API Call which gets the images in the results variable

In [13]:
from PIL import Image
import requests
import cv2
import numpy as np
from urllib.request import urlopen

import xml.etree.ElementTree as ET
import flickrapi

api_key = u'efb284edc88f1563e0e398e0d42ca677'
api_secret = u'da3cfff301f5ea38'

#200368767@N08 my user ID

flickr = flickrapi.FlickrAPI(api_key, api_secret)
flickr.authenticate_via_browser(perms='read')

flickr = flickrapi.FlickrAPI(api_key, api_secret, format='etree')
url_type = 'url_n'

def ShowWindow(image, title, i, row) -> int:

    #show image, move it to position, and increment i
    cv2.imshow(f'{title}: {row},{i}', image)
    cv2.moveWindow(f'{title}: {row},{i}', (i%8)*250, row*250)

    return i+1

def ShowWindowInMain(canvas, image, i) -> int:

    if(i == 0):
        # Place the first image on the left side of the canvas
        canvas[125:375, 100:350] = image
    else:
        # Place the second image on the right side of the canvas
        canvas[125:375, 400:650] = image

    cv2.imshow('Main Window', canvas)


Support method for Showing an image

Unused Testing

In [ ]:
column = 0
for photo in results:

    row = 0
    for data in photo:
        
        title = data.get('title')
        url = data.get(url_type)

        if(url != None):

            image_raw = PostProcesser.GetImageFromUrl(url)
            #decode the image with color
            image = cv2.imdecode(image_raw, cv2.IMREAD_COLOR)
            #resize the image
            image = cv2.resize(image, (250, 250))

            column = ShowWindow(image, "original", column, row) 

            image = PostProcesser.GetNormalizedImg(image)
            #column = ShowWindow(image, "normalized", column, row)

            imageKMeans = cv2.add(PostProcesser.GetKMeanImg(image, num_clusters = 6), 25)
            column = ShowWindow(imageKMeans, "KMEAN", column, row)

            imageSkeleton = PostProcesser.GetSkeletonImg(imageKMeans, kernelSize = 3, _iterations = 1)
            column = ShowWindow(imageSkeleton, "skeleton", column, row)

            denoisedImg = PostProcesser.GetDenoisedImg(imageKMeans)
            column = ShowWindow(denoisedImg, "denoised", column, row)   

            imageDeNoisedGray = PostProcesser.GetGrayscaleResizedImg(denoisedImg)
            #column = ShowWindow(imageDeNoisedGray, "grayscaled", column, row)        

            imageGraySkeleton = PostProcesser.GetSkeletonImg(imageDeNoisedGray, kernelSize = 5, _iterations = 1)
            column = ShowWindow(imageGraySkeleton, "graySkel", column, row)     
            
            imageGraySkeletonBlurred = PostProcesser.GetBlurredImg(imageGraySkeleton, 19)
            #column = ShowWindow(imageGraySkeletonBlurred, "graySkelBlur", column, row)   

            processedImage = cv2.cvtColor(imageGraySkeletonBlurred, cv2.COLOR_GRAY2BGR)
            processedImage = cv2.drawContours(processedImage, PostProcesser.GetContours(processedImage, False, threshold1 = 30, threshold2 = 120), -1, (0, 255, 255), 1)
            column = ShowWindow(processedImage, "processed1", column, row) 
            
            processedImage2 = cv2.cvtColor(PostProcesser.GetBlurredImg(imageDeNoisedGray, 19), cv2.COLOR_GRAY2BGR)
            processedImage2 = cv2.drawContours(processedImage2, PostProcesser.GetContours(processedImage2, False, threshold1 = 30, threshold2 = 120), -1, (0, 0, 255), 1)
            column = ShowWindow(processedImage2, "processed2", column, row) 
            



            #cv2.bitwise_or(image, postProcesser.GetEdges(image_raw), image)
            #processedImage = cv2.drawContours(image, postProcesser.GetContours(image, False), -1, (0, 0, 255), 1)
            #morph = cv2.drawContours(image, postProcesser.GetContours(image, True), -1, (0, 255, 0), 1)
            
        print()
        row += 1
        column = 0

cv2.waitKey(0)
cv2.destroyAllWindows()

Prosessing of a single image (this is where the magic happens):


for blob coloring first I tried cv2.drawContours() with thickness=cv2.FILLED, it didnt work as I Wanted
then cv2.fillpoly and cv2.fillpolyConvex, not working as intented as it didnt color all the blobs

then in the end what worked is just regular floodfill, but since the contours are not giving me all the blobs,
we always pick a part of the image which isnt colored yet, color it, and repeat


for every color blob, when deciding the color, im retrieving:
    the mean color of the area filled
    the center of the area
    the position of uppermost pixel of the area
    the posiiton of the lowermost pixel of the area

HARD RULES
    if its connected to the ceiling it must be sky (the mountain might also be connected to the ceiling, so to decide which blob is the sky)
    if its not connected to the ceiling it cannot be sky no matter what
    if its connected to the bottom and its white it must by snow

WEAK RULES
    if its in the upper 70-90% it has a higher likelyhood of becoming snow (less white is also considered snow)    
    if its in the middle it has higher probability of being either green/forest or brown/mountain


In [14]:
import cv2
from cv2.typing import MatLike, Point, Size
import numpy as np
from Processer import PostProcesser

def ProcessAndShow(image:MatLike, column, row, centerized:bool=True):
    
    column = ShowWindow(image, "original", column, row) 

    gray = PostProcesser.GetGrayscaleResizedImg(image)  
    imageDeNoised = PostProcesser.GetDenoisedImg(image)
    blurred = PostProcesser.GetBlurredImg(image, 17)
    blurredDeNoised = PostProcesser.GetBlurredImg(imageDeNoised, 17)

    illumination = PostProcesser.GetNormalizedIllumination(image)
    column = ShowWindow(illumination, "Illumination", column, row) 

    imageDeNoised2 = PostProcesser.GetDenoisedImg(illumination)
    blurred2 = PostProcesser.GetBlurredImg(illumination, 17)
    blurredDeNoised2 = PostProcesser.GetBlurredImg(imageDeNoised2, 17)

    threshold1 = 25
    threshold2 = 100
    
    quantizedImg = PostProcesser.GetKMeanImg(image, 4)
    quantizedBlurred = PostProcesser.GetKMeanImg(blurred, 4)
    quantizedDen = PostProcesser.GetKMeanImg(imageDeNoised, 4)
    quantizedBlurredDen = PostProcesser.GetKMeanImg(blurredDeNoised, 4)
    
    #column = ShowWindow(quantized, "quantized", column, row)   



    processedImage = PostProcesser.GetContours(blurred, blurred, True, threshold1, threshold2, shape=cv2.MORPH_ELLIPSE, strength=(11,11))
    column = ShowWindow(processedImage, "P_Blurred", column, row) 

    processedImage2 = PostProcesser.GetContours(blurredDeNoised, blurredDeNoised, True, threshold1, threshold2, shape=cv2.MORPH_ELLIPSE, strength=(11,11))
    column = ShowWindow(processedImage2, "P_blurredDeN", column, row) 
    


    #processedImage4 = PostProcesser.GetContours(blurred, blurred, True, threshold1, threshold2, shape=cv2.MORPH_ELLIPSE, strength=(21,21))
    #column = ShowWindow(processedImage4, "P_Blurred2", column, row) 

    #processedImage5 = PostProcesser.GetContours(blurredDeNoised, blurredDeNoised, True, threshold1, threshold2, shape=cv2.MORPH_ELLIPSE, strength=(21,21))
    #column = ShowWindow(processedImage5, "P_blurredDeN2", column, row) 



    '''
    #using illumation-normalized image test

    processedImage4 = PostProcesser.GetContours(blurred2, blurred2, True, threshold1, threshold2, connectWeak = False)
    column = ShowWindow(processedImage4, "P_Blurred2", column, row) 

    processedImage5 = PostProcesser.GetContours(imageDeNoised2, imageDeNoised2, True, threshold1, threshold2, connectWeak = False)
    column = ShowWindow(processedImage5, "P_Denoised2", column, row) 

    processedImage6 = PostProcesser.GetContours(blurredDeNoised2, blurredDeNoised2, True, threshold1, threshold2, connectWeak = False)
    column = ShowWindow(processedImage6, "P_blurredDeN2", column, row) 
    '''



    #CONNECTING WEAK LINES TEST
    #processedImage4 = PostProcesser.GetContours(blurred, blurred, True, threshold1, threshold2, connectWeak = True)
    #column = ShowWindow(processedImage4, "P_Blurred2", column, row) 

    #processedImage5 = PostProcesser.GetContours(imageDeNoised, imageDeNoised, True, threshold1, threshold2, connectWeak = True)
    #column = ShowWindow(processedImage5, "P_Denoised2", column, row) 

    #processedImage6 = PostProcesser.GetContours(blurredDeNoised, blurredDeNoised, True, threshold1, threshold2, connectWeak = True)
    #column = ShowWindow(processedImage6, "P_blurredDeN2", column, row) 




    '''
    processedImage4 = PostProcesser.GetContours(quantizedBlurred, quantizedBlurred, True, threshold1, threshold2, 2)
    column = ShowWindow(processedImage4, "P_KBlurred", column, row)   

    processedImage5 = PostProcesser.GetContours(quantizedDen, quantizedDen, True, threshold1, threshold2, 2)
    column = ShowWindow(processedImage5, "P_KDen", column, row)   

    processedImage6 = PostProcesser.GetContours(quantizedBlurredDen, quantizedBlurredDen, True, threshold1, threshold2, 2)
    column = ShowWindow(processedImage6, "P_KBlurredDen", column, row)   
    '''

    #processedImage = PostProcesser.GetContours(blurred, blurred, False, threshold1, threshold2, 2)
    #column = ShowWindow(processedImage, "P_Blurred2", column, row) 

    #processedImage2 = PostProcesser.GetContours(imageDeNoised, imageDeNoised, False, 20, 200, 2)
    #column = ShowWindow(processedImage2, "P_Denoised2", column, row) 
    
    #processedImage = postProcesser.GetBinaryContoured(image, gray, thresholded)
    #processedImage3 = PostProcesser.GetContours(blurredDeNoised, blurredDeNoised, False, threshold1, threshold2, 2)
    #column = ShowWindow(processedImage3, "P_blurredDen2", column, row) 

Main method:

In [15]:
from Processer import PostProcesser
import random

#print("result length", len(results[0]))
def ShowImages():
    results = flickr.photos.search(page='1', tag_mode='all', tags="mountain, mountains, Mountain, Mountains green, nature, landscape, alps, snow", extras=url_type, per_page=1, sort = 'relevance', geo_context = '0', content_types = '0')
    try:
        if(len(results[0]) > 0):
            column = 0
            for photo in results:
                row = 0
                for data in photo:
                    title = data.get('title')
                    url = data.get(url_type)

                    if(url != None):

                        image_raw = PostProcesser.GetImageFromUrl(url)
                        #decode the image with color
                        image = cv2.imdecode(image_raw, cv2.IMREAD_COLOR)
                        #resize the image
                        image = cv2.resize(image, (250, 250))

                        ProcessAndShow(image, column, row)



                        row += 1
                        column = 0
        else:
            image = PostProcesser.GetImageFromPath("C:\\mountain1.jpg")
            ProcessAndShow(image, 0, 0)   
    except:
        image = PostProcesser.GetImageFromPath("C:\\mountain1.jpg")
        ProcessAndShow(image, 0, 0)

def ShowImagesInMain(canvas:MatLike):
    results = flickr.photos.search(page=random.randint(1, 500), tag_mode='all', tags="mountain, mountains, Mountain, Mountains green, nature, landscape, alps, snow", extras=url_type, per_page=1, sort = 'relevance', geo_context = '0', content_types = '0')
    try:
        if(len(results[0]) > 0):
            column = 0
            for photo in results:
                row = 0
                for data in photo:
                    title = data.get('title')
                    url = data.get(url_type)

                    if(url != None):

                        image_raw = PostProcesser.GetImageFromUrl(url)
                        #decode the image with color
                        image = cv2.imdecode(image_raw, cv2.IMREAD_COLOR)
                        #resize the image
                        image = cv2.resize(image, (250, 250))

                        ProcessAndShowInMain(canvas, image)

                        row += 1
                        column = 0
        else:
            image = PostProcesser.GetImageFromPath("C:\\mountain1.jpg")
            ProcessAndShowInMain(canvas, image)   
    except:
        image = PostProcesser.GetImageFromPath("C:\\mountain1.jpg")
        ProcessAndShowInMain(canvas, image)


MAIN

In [16]:
import cv2
from cv2.typing import MatLike, Point, Size
import numpy as np
from Processer import PostProcesser

def ProcessAndShowInMain(canvas:MatLike, image:MatLike):
    
    ShowWindowInMain(canvas, image, 0)
 
    imageDeNoised = PostProcesser.GetDenoisedImg(image)
    blurred = PostProcesser.GetBlurredImg(image, 15)
    blurredDeNoised = PostProcesser.GetBlurredImg(imageDeNoised, 15)

    threshold1 = 25
    threshold2 = 50
    
    processedImage = PostProcesser.GetContours(blurred, blurred, True, threshold1, threshold2, shape=cv2.MORPH_ELLIPSE, strength=(11,11))
    ShowWindowInMain(canvas, processedImage, 1)

In [17]:

def PlaceButton(canvas, YPos:int):
    # Define the button properties
    button_color = (100, 255, 100)  # Red button color
    button_text = "Next"  # Text on the button
    text_color = (0, 0, 0)  # White text color
    font = cv2.FONT_HERSHEY_SIMPLEX  # Font for the text
    font_scale = 1  # Font scale
    thickness = 2  # Thickness of the text

    # Calculate text size to position it properly
    text_size = cv2.getTextSize(button_text, font, font_scale, thickness)
    text_width = text_size[0][0]
    text_height = text_size[0][1]


    # Calculate button position and size
    button_width = text_width + 40  # Add some padding
    button_height = text_height + 20  # Add some padding
    button_x = (canvas.shape[1] - button_width) // 2
    #button_y = (canvas[0] - button_height) // 2
    button_y = YPos

    # Draw the button rectangle
    cv2.rectangle(canvas, (button_x, button_y), (button_x + button_width, button_y + button_height), button_color, -1)

    # Calculate text position
    text_x = button_x + (button_width - text_width) // 2
    text_y = button_y + (button_height + text_height) // 2

    # Add the text on the button
    cv2.putText(canvas, button_text, (text_x, text_y), font, font_scale, text_color, thickness)

    '''
    # Draw the button rectangle
    cv2.rectangle(image, (TopLeftPos[0], TopLeftPos[1]), (TopLeftPos[0] + button_width, TopLeftPos[1] + button_height), button_color, -1)

    # Calculate text position
    text_x = TopLeftPos[0] + (button_width - text_width) // 2
    text_y = TopLeftPos[1] + (button_height + text_height) // 2

    # Add the text on the button
    cv2.putText(image, button_text, (text_x, text_y), font, font_scale, text_color, thickness)
    
    '''

    
    # Callback function to handle mouse events
    def on_mouse_click(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            # Check if the click is within the button region
            if button_x <= x <= button_x + button_width and button_y <= y <= button_y + button_height:
                #print("Button clicked!")
                ShowImagesInMain(canvas)
                
    cv2.setMouseCallback('Main Window', on_mouse_click)


In [18]:
import cv2
import ctypes
import numpy as np
from pynput.keyboard import Key, Listener

# Create a larger canvas to hold both images
canvas_width = 750
canvas_height = 500
canvas = 150 * np.ones((canvas_height, canvas_width, 3), dtype=np.uint8)  # White canvas

blackImg = 50* np.ones((250, 250, 3), dtype=np.uint8)


OrigImgPos = (125, 375, 100, 350)
ProcessedImgPos = (125, 375, 400, 650)


# Place the first image on the left side of the canvas
canvas[125:375, 100:350] = blackImg

# Place the second image on the right side of the canvas
canvas[125:375, 400:650] = blackImg

cv2.namedWindow('Main Window')

PlaceButton(canvas, 415)

cv2.imshow('Main Window', canvas)



user32 = ctypes.windll.user32
screen_width = user32.GetSystemMetrics(0)
screen_height = user32.GetSystemMetrics(1)

x_pos = (screen_width - canvas_width) // 2
y_pos = (screen_height - canvas_height - 200) // 2

cv2.moveWindow('Main Window', x_pos, y_pos)




cv2.waitKey(0)
cv2.destroyAllWindows()
